In [16]:
import json
from datetime import datetime

In [17]:
resfile = 'result/gcb_0.json'

with open(resfile) as jf:
    RES = json.load(jf)

print(len(RES))

118


In [78]:
def dump1(path):
    with open(path) as jf:
        RES = json.load(jf)    

    lastT = {}
    dur = []
    for FR in RES:
        fn = FR['Frame']
        #print(f'{fn}')
        for BC in FR['GCB']:
            bc = BC['type']
            if bc == 'CL-Beacon': continue
            expTstr = BC['expT']
            expT = datetime.strptime(expTstr, "%Y-%m-%dT%H:%M:%S.%f")
            dT   = BC['dT']
            if bc == 'CL-Beacon':
                ratio = BC['result']['time_cl'][0]
            elif bc == 'CM-Beacon':
                ratio = BC['result']['time_cm'][0]
            else:
                ratio = 0
            print(f'{fn}\t{bc} {expT}({round(dT*1000,2)}ms)  ratio={ratio} ',end='\t')
            try:
                ds  = round((expT - lastT[bc]).total_seconds() * 1000,2)
                print(f'{ds}')
                dur.append(ds)
            except KeyError:
                print()
            lastT[bc] = expT
            
            
    durAve = sum(dur) / len(dur)
    durVar = sum((x - durAve) ** 2 for x in dur) / len(dur)
    print(f'Ave={round(durAve,3)},fps={round(1000/durAve,2)} , Var={round(durVar,3)}')

In [79]:
#dump1('result/gcb_0.json')
dump1('result/HRC33_lap29.json')
#dump1('result/i15_DL_1x_H_down.json')
#dump1('result/JSB_S_lap2.json')

0	CM-Beacon 2023-08-06 12:31:03.091780(0.59ms)  ratio=0.955 	
1	CM-Beacon 2023-08-06 12:31:03.095730(0.59ms)  ratio=0.955 	3.95
2	CM-Beacon 2023-08-06 12:31:03.099770(0.59ms)  ratio=0.964 	4.04
3	CM-Beacon 2023-08-06 12:31:03.103720(0.59ms)  ratio=0.911 	3.95
4	CM-Beacon 2023-08-06 12:31:03.108260(0.42ms)  ratio=0.92 	4.54
5	CM-Beacon 2023-08-06 12:31:03.112720(0.59ms)  ratio=0.955 	4.46
6	CM-Beacon 2023-08-06 12:31:03.116750(0.59ms)  ratio=0.964 	4.03
7	CM-Beacon 2023-08-06 12:31:03.120710(0.59ms)  ratio=0.964 	3.96
8	CM-Beacon 2023-08-06 12:31:03.133230(0.42ms)  ratio=0.946 	12.52
9	CM-Beacon 2023-08-06 12:31:03.137270(0.42ms)  ratio=0.938 	4.04
10	CM-Beacon 2023-08-06 12:31:03.141730(0.59ms)  ratio=0.946 	4.46
11	CM-Beacon 2023-08-06 12:31:03.145770(0.59ms)  ratio=0.946 	4.04
12	CM-Beacon 2023-08-06 12:31:03.158210(0.42ms)  ratio=0.938 	12.44
13	CM-Beacon 2023-08-06 12:31:03.162750(0.59ms)  ratio=0.955 	4.54
14	CM-Beacon 2023-08-06 12:31:03.166700(0.59ms)  ratio=0.964 	3.95
15	CM-Be

In [81]:
import numpy as np
from sklearn.linear_model import LinearRegression

# サンプルデータ（単調増加する数値の配列）
d = [1, 3, 6, 10, 12, 18, 21]

# 初期増分を計算
initial_increment = d[1] - d[0]

# 増分を推定する
def estimate_increment(data, initial_increment):
    increments = [initial_increment]
    for i in range(2, len(data)):
        increments.append(data[i] - data[i-1])

    # フィルター処理
    valid_increments = [inc for inc in increments if 2 <= inc <= 4]

    # 推定された増分を取得
    estimated_increment = np.mean(valid_increments)
    return estimated_increment

# 欠損箇所を埋める
def fill_missing_values(data, estimated_increment):
    for i in range(2, len(data)):
        if data[i] is None:
            data[i] = data[i-1] + estimated_increment
    return data

# 推定された増分を計算
estimated_increment = estimate_increment(d, initial_increment)

# 欠損箇所を埋めてデータを取得
filled_data = fill_missing_values(d, estimated_increment)

# データのインデックス
indices = list(range(len(filled_data)))

# NumPyのarrayに変換
X = np.array(indices).reshape(-1, 1)
y = np.array(filled_data)

# 線形回帰モデルを構築
model = LinearRegression().fit(X, y)

# 傾きと切片を取得
slope = model.coef_[0]
intercept = model.intercept_

# 結果を表示
print(f"Slope (傾き): {slope}")
print(f"Intercept (切片): {intercept}")


Slope (傾き): 3.4285714285714275
Intercept (切片): -0.14285714285714057


In [85]:
from datetime import datetime, timedelta
import numpy as np
from sklearn.linear_model import LinearRegression

def perform_linear_regression(timestamps):
    """
    Perform linear regression on the given timestamps.

    Args:
    timestamps (list of datetime): List of timestamps for each frame.

    Returns:
    LinearRegression model: A fitted linear regression model.
    """
    # Convert timestamps to a format suitable for regression
    # Here, we are using the total number of seconds since the first timestamp
    # as our independent variable
    start_time = timestamps[0]
    X = np.array([(ts - start_time).total_seconds() for ts in timestamps]).reshape(-1, 1)
    
    # As the dependent variable, we can use microseconds of each timestamp
    y = np.array([ts.microsecond for ts in timestamps])

    # Create and fit the linear regression model
    model = LinearRegression()
    model.fit(X, y)

    return model

# Example usage
timestamps = [datetime(2024, 1, 1, 12, 0, 0, microsecond=i*33333) for i in range(30)]
model = perform_linear_regression(timestamps)

# Example prediction
X_new = np.array([(datetime(2024, 1, 1, 12, 1, 0) - timestamps[0]).total_seconds()]).reshape(-1, 1)
predicted_microseconds = model.predict(X_new)
predicted_microseconds


array([59999999.99999999])

In [94]:
def generate_realistic_test_data_v2(total_frames, max_missing_frames=3, base_interval_ms=33.33):
    """
    Generate realistic test data for timestamps and frame numbers considering errors and frame drops.
    Large error does not affect the subsequent timestamps.

    Args:
    total_frames (int): Total number of frames to simulate.
    max_missing_frames (int): Maximum number of consecutive missing frames.
    base_interval_ms (float): Base interval between frames in milliseconds.

    Returns:
    tuple: Two lists - timestamps and corresponding frame numbers.
    """
    timestamps = []
    frame_numbers = []
    current_time = datetime(2024, 1, 1, 12, 0, 0)
    frame_counter = 0

    for _ in range(total_frames):
        # Add base interval with some jitter (2ms)
        jitter = timedelta(milliseconds=2 * random.choice([-1, 1]))
        current_time += timedelta(milliseconds=base_interval_ms) + jitter

        # Simulate frame dropping
        if random.random() < 0.1:  # Assuming 10% chance for frame drop
            num_missing_frames = random.randint(1, max_missing_frames)
            current_time += timedelta(milliseconds=base_interval_ms * num_missing_frames)
            frame_counter += num_missing_frames

        # Add large error (±500ms) with 10% probability, but do not affect current_time
        large_error = timedelta(milliseconds=0)
        if random.random() < 0.1:
            large_error = timedelta(milliseconds=random.uniform(-500, 500))

        timestamps.append(current_time + large_error)
        frame_numbers.append(frame_counter)
        frame_counter += 1

    return timestamps, frame_numbers

# Generate test data with the modified behavior
total_frames = 100
timestamps_v2, frame_numbers_v2 = generate_realistic_test_data_v2(total_frames)

# Display the first few timestamps and frame numbers for verification
timestamps_v2[:5], frame_numbers_v2[:5]


([datetime.datetime(2024, 1, 1, 12, 0, 0, 101990),
  datetime.datetime(2024, 1, 1, 12, 0, 0, 166650),
  datetime.datetime(2024, 1, 1, 12, 0, 0, 197980),
  datetime.datetime(2024, 1, 1, 12, 0, 0, 229310),
  datetime.datetime(2024, 1, 1, 12, 0, 0, 260640)],
 [2, 4, 5, 6, 7])

In [95]:

def perform_linear_regression_for_frames(frame_numbers, timestamps):
    """
    Perform linear regression using frame numbers and timestamps.

    Args:
    frame_numbers (list of int): Frame numbers.
    timestamps (list of datetime): Timestamps corresponding to frame numbers.

    Returns:
    LinearRegression model: A fitted linear regression model.
    """
    # Prepare data for regression
    X = np.array(frame_numbers).reshape(-1, 1)
    y = np.array([(ts - timestamps[0]).total_seconds() for ts in timestamps])

    # Perform linear regression
    model = LinearRegression()
    model.fit(X, y)

    return model

def predict_frame_timestamp(frame_number, model, start_timestamp):
    """
    Predict the timestamp for a given frame number using the regression model.

    Args:
    frame_number (int): The frame number to predict the timestamp for.
    model (LinearRegression): The fitted linear regression model.
    start_timestamp (datetime): The timestamp of the first frame.

    Returns:
    datetime: Predicted timestamp for the given frame number.
    """
    predicted_seconds = model.predict(np.array([frame_number]).reshape(-1, 1))[0]
    predicted_timestamp = start_timestamp + timedelta(seconds=predicted_seconds)
    return predicted_timestamp

# Example usage
total_frames = 100
timestamps, frame_numbers = generate_realistic_test_data_v2(total_frames)

generate_realistic_test_data
model = perform_linear_regression_for_frames(frame_numbers, timestamps)

# Predict timestamp for a specific frame number
frame_number_to_predict = 25
predicted_timestamp = predict_frame_timestamp(frame_number_to_predict, model, timestamps[0])
predicted_timestamp



datetime.datetime(2024, 1, 1, 12, 0, 0, 848841)

In [96]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool

# Prepare data for plotting
predicted_timestamps = [predict_frame_timestamp(fn, model, timestamps[0]) for fn in frame_numbers]
predicted_seconds = [(ts - timestamps[0]).total_seconds() for ts in predicted_timestamps]

# Convert to milliseconds for easier visualization
original_milliseconds = [(ts - timestamps[0]).total_seconds() * 1000 for ts in timestamps]
predicted_milliseconds = [sec * 1000 for sec in predicted_seconds]

# Prepare data for regression line
regression_line_x = range(min(frame_numbers), max(frame_numbers) + 1)
regression_line_y = [model.predict([[x]])[0] * 1000 for x in regression_line_x]

# Coefficients of the regression line
slope = model.coef_[0]
intercept = model.intercept_

# Create a Bokeh plot
output_notebook()  # Output to notebook
p = figure(title="Frame Timestamp Estimation", x_axis_label='Frame Number', y_axis_label='Timestamp (milliseconds)',width=1536,height=512)
p.circle(frame_numbers, original_milliseconds, size=8, color="blue", alpha=0.5, legend_label="Original Timestamps")
p.circle(frame_numbers, predicted_milliseconds, size=8, color="red", alpha=0.5, legend_label="Predicted Timestamps")
p.line(regression_line_x, regression_line_y, color="green", legend_label="Regression Line")

# Add hover tool
hover = HoverTool()
hover.tooltips=[('Frame Number', '@x'), ('Timestamp (ms)', '@y')]
p.add_tools(hover)

# Show plot
show(p)

# Print coefficients
slope, intercept



Loading BokehJS ...

(0.03349799732275746, -0.01993898036991304)

In [97]:
# Perform linear regression with the new test data version 2
model_v2 = perform_linear_regression_for_frames(frame_numbers_v2, timestamps_v2)

# Prepare data for plotting
predicted_timestamps_v2 = [predict_frame_timestamp(fn, model_v2, timestamps_v2[0]) for fn in frame_numbers_v2]
predicted_seconds_v2 = [(ts - timestamps_v2[0]).total_seconds() for ts in predicted_timestamps_v2]

# Convert to milliseconds for easier visualization
original_milliseconds_v2 = [(ts - timestamps_v2[0]).total_seconds() * 1000 for ts in timestamps_v2]
predicted_milliseconds_v2 = [sec * 1000 for sec in predicted_seconds_v2]

# Calculate the difference between the actual and predicted timestamps
difference_milliseconds = [orig - pred for orig, pred in zip(original_milliseconds_v2, predicted_milliseconds_v2)]

# Coefficients of the regression line
slope_v2 = model_v2.coef_[0]
intercept_v2 = model_v2.intercept_

# Create a Bokeh plot
p_v2 = figure(title="Frame Timestamp Estimation Error", x_axis_label='Frame Number', y_axis_label='Error (milliseconds)')
p_v2.circle(frame_numbers_v2, difference_milliseconds, size=8, color="red", alpha=0.5, legend_label="Error (Original - Predicted)")

# Add hover tool
hover_v2 = HoverTool()
hover_v2.tooltips=[('Frame Number', '@x'), ('Error (ms)', '@y')]
p_v2.add_tools(hover_v2)

# Show plot
show(p_v2)

# Print regression line equation
f"Regression Line Equation: y = {slope_v2:.5f}x + {intercept_v2:.5f}"



'Regression Line Equation: y = 0.03307x + -0.06763'

In [98]:
from bokeh.plotting import figure

def plot_frame_timestamp_estimation_error(timestamps, figWidth=800, figHeight=400):
    """
    Plot the frame timestamp estimation error using the given timestamps.

    Args:
    timestamps (list of datetime): List of timestamps for each frame.
    figWidth (int): Width of the figure.
    figHeight (int): Height of the figure.

    Returns:
    figure: Bokeh figure object showing the estimation error.
    """
    # Generate frame numbers for the given timestamps
    frame_numbers = list(range(len(timestamps)))

    # Perform linear regression
    model = perform_linear_regression_for_frames(frame_numbers, timestamps)

    # Predict timestamps
    predicted_timestamps = [predict_frame_timestamp(fn, model, timestamps[0]) for fn in frame_numbers]
    predicted_seconds = [(ts - timestamps[0]).total_seconds() for ts in predicted_timestamps]

    # Convert to milliseconds for easier visualization
    original_milliseconds = [(ts - timestamps[0]).total_seconds() * 1000 for ts in timestamps]
    predicted_milliseconds = [sec * 1000 for sec in predicted_seconds]

    # Calculate the difference between the actual and predicted timestamps
    difference_milliseconds = [orig - pred for orig, pred in zip(original_milliseconds, predicted_milliseconds)]

    # Create a Bokeh plot
    p = figure(title="Frame Timestamp Estimation Error", x_axis_label='Frame Number', y_axis_label='Error (milliseconds)',
               width=figWidth, height=figHeight)
    p.circle(frame_numbers, difference_milliseconds, size=8, color="red", alpha=0.5)

    return p

# Example usage of the function
p_example = plot_frame_timestamp_estimation_error(timestamps_v2, 800, 400)
show(p_example)  # Show the plot
p_example  # Return the figure object


Figure(id='1967', ...)

In [108]:
def estimate_frame_numbers(timestamps, max_missing_frames=3):
    """
    Estimate frame numbers considering possible frame drops.

    Args:
    timestamps (list of datetime): List of timestamps for each frame.
    max_missing_frames (int): Maximum number of consecutive missing frames.

    Returns:
    list: Estimated frame numbers.
    """
    estimated_frame_numbers = []
    base_interval = (timestamps[-1] - timestamps[0]) / (len(timestamps) - 1)
    
    for i in range(len(timestamps)):
        if i == 0:
            estimated_frame_numbers.append(0)
        else:
            interval = timestamps[i] - timestamps[i-1]
            frame_jump = round(interval / base_interval)
            estimated_frame_numbers.append(estimated_frame_numbers[i-1] + frame_jump)

    return estimated_frame_numbers

def plot_frame_timestamp_estimation_error_v2(timestamps, figWidth=800, figHeight=400):
    """
    Plot the frame timestamp estimation error using the given timestamps.
    Frame numbers are estimated considering possible frame drops.

    Args:
    timestamps (list of datetime): List of timestamps for each frame.
    figWidth (int): Width of the figure.
    figHeight (int): Height of the figure.

    Returns:
    figure: Bokeh figure object showing the estimation error.
    """
    # Estimate frame numbers
    frame_numbers = estimate_frame_numbers(timestamps)

    # Perform linear regression
    model = perform_linear_regression_for_frames(frame_numbers, timestamps)

    # Predict timestamps
    predicted_timestamps = [predict_frame_timestamp(fn, model, timestamps[0]) for fn in frame_numbers]
    predicted_seconds = [(ts - timestamps[0]).total_seconds() for ts in predicted_timestamps]

    # Convert to milliseconds for easier visualization
    original_milliseconds = [(ts - timestamps[0]).total_seconds() * 1000 for ts in timestamps]
    predicted_milliseconds = [sec * 1000 for sec in predicted_seconds]

    # Calculate the difference between the actual and predicted timestamps
    difference_milliseconds = [orig - pred for orig, pred in zip(original_milliseconds, predicted_milliseconds)]

    # Create a Bokeh plot
    p = figure(title="Frame Timestamp Estimation Error", x_axis_label='Frame Number', y_axis_label='Error (milliseconds)',
               width=figWidth, height=figHeight)
    p.circle(frame_numbers, difference_milliseconds, size=8, color="red", alpha=0.5)

    return p

# Example usage of the function with the modified test data
p_modified_example = plot_frame_timestamp_estimation_error_v2(timestamps_v2, 800, 400)
show(p_modified_example)  # Show the plot
#p_modified_example  # Return the figure object



In [107]:
frames = estimate_frame_numbers(timestamps_v2)
lastT = False
for fr,T in zip(frames,timestamps_v2):
    if lastT:
        print(fr,T, (T-lastT).total_seconds())
    lastT = T

2 2024-01-01 12:00:00.166650 0.06466
3 2024-01-01 12:00:00.197980 0.03133
4 2024-01-01 12:00:00.229310 0.03133
5 2024-01-01 12:00:00.260640 0.03133
6 2024-01-01 12:00:00.295970 0.03533
7 2024-01-01 12:00:00.331300 0.03533
8 2024-01-01 12:00:00.366630 0.03533
9 2024-01-01 12:00:00.397960 0.03133
10 2024-01-01 12:00:00.429290 0.03133
11 2024-01-01 12:00:00.460620 0.03133
12 2024-01-01 12:00:00.495950 0.03533
15 2024-01-01 12:00:00.597940 0.10199
16 2024-01-01 12:00:00.629270 0.03133
17 2024-01-01 12:00:00.664600 0.03533
20 2024-01-01 12:00:00.762590 0.09799
21 2024-01-01 12:00:00.793920 0.03133
22 2024-01-01 12:00:00.825250 0.03133
23 2024-01-01 12:00:00.856580 0.03133
24 2024-01-01 12:00:00.887910 0.03133
23 2024-01-01 12:00:00.844397 -0.043513
26 2024-01-01 12:00:00.954570 0.110173
27 2024-01-01 12:00:00.985900 0.03133
28 2024-01-01 12:00:01.017230 0.03133
29 2024-01-01 12:00:01.048560 0.03133
30 2024-01-01 12:00:01.079890 0.03133
31 2024-01-01 12:00:01.111220 0.03133
32 2024-01-01 12:

In [109]:
from bokeh.plotting import figure, show

p = figure(plot_width=400, plot_height=400)
p.segment(x0=[1, 2, 3], y0=[1, 2, 3], x1=[1, 2, 3], y1=[2, 3, 4], color="green", line_width=2)

show(p)


In [ ]:
import numpy as np

def improved_estimate_frame_numbers(timestamps):
    """
    Improved estimation of frame numbers considering possible frame drops.
    Estimates the base interval by examining the frequency of differences between adjacent timestamps.

    Args:
    timestamps (list of datetime): List of timestamps for each frame.

    Returns:
    list: Estimated frame numbers.
    """
    # Calculate differences between adjacent timestamps in seconds
    time_diffs = [(timestamps[i] - timestamps[i - 1]).total_seconds() for i in range(1, len(timestamps))]

    # Group the time differences and find the most frequent group
    hist, bin_edges = np.histogram(time_diffs, bins='auto')
    most_frequent_bin_index = np.argmax(hist)
    base_interval = (bin_edges[most_frequent_bin_index] + bin_edges[most_frequent_bin_index + 1]) / 2

    # Estimate frame numbers based on the most frequent time difference
    estimated_frame_numbers = [0]
    for diff in time_diffs:
        frame_jump = round(diff / base_interval)
        estimated_frame_numbers.append(estimated_frame_numbers[-1] + frame_jump)

    return estimated_frame_numbers

# Example usage
# Assuming timestamps_v2 is the list of datetime objects
# estimated_frame_numbers = improved_estimate_frame_numbers(timestamps_v2)
# estimated_frame_numbers[:10]  # Display first 10 estimated frame numbers



In [110]:
from datetime import datetime,timedelta
from typing import List
import numpy as np
from scipy.stats import linregress

def estimate_frame_numbers(exptime: List[datetime], fps: float) -> List[int]:
    """Estimate the frame number for each datetime in exptime."""
    base_time = exptime[0]
    frame_numbers = [(exp_time - base_time).total_seconds() * fps for exp_time in exptime]
    return frame_numbers

def filter_abnormal_values(frame_numbers: List[int], exptime: List[datetime], fps: float) -> (List[int], List[datetime]):
    """Filter out abnormal values based on fps and frame numbers."""
    filtered_frame_numbers = []
    filtered_exptime = []
    for i in range(len(frame_numbers)):
        # Exclude abnormal values (where the frame number is too far off or goes backward)
        if i > 0 and (frame_numbers[i] < frame_numbers[i-1] or frame_numbers[i] - frame_numbers[i-1] > 4 * fps):
            continue
        filtered_frame_numbers.append(frame_numbers[i])
        filtered_exptime.append(exptime[i])
    return filtered_frame_numbers, filtered_exptime

def linear_regression_with_fixed_slope(frame_numbers: List[int], exptime: List[datetime], fps: float) -> float:
    """Perform linear regression with a fixed slope of 1/fps and return the y-intercept."""
    # Convert datetime to seconds since the first timestamp for regression analysis
    times_in_seconds = [(exp_time - exptime[0]).total_seconds() for exp_time in exptime]
    slope = 1 / fps
    intercept = np.mean(times_in_seconds) - slope * np.mean(frame_numbers)
    return intercept

In [110]:
# Example usage
fps = 30  # frames per second
# Assume exptime is a list of datetime objects, for example:
# exptime = [datetime(2020, 1, 1, 12, 0, 0), datetime(2020, 1, 1, 12, 0, 1), ...]



# Correcting the missing import and re-running the example

# Example data (this should be replaced with the actual datetime objects)
exptime_example = [datetime(2020, 1, 1) + timedelta(seconds=i*1/fps) for i in range(60)]  # 2 seconds of frames

# Estimate frame numbers
frame_numbers = estimate_frame_numbers(exptime_example, fps)

# Filter abnormal values
filtered_frame_numbers, filtered_exptime = filter_abnormal_values(frame_numbers, exptime_example, fps)

# Perform linear regression with fixed slope
y_intercept = linear_regression_with_fixed_slope(filtered_frame_numbers, filtered_exptime, fps)

y_intercept

-1.1102230246251565e-16

In [112]:
from datetime import datetime, timedelta
from typing import List
import numpy as np

def normalize_and_filter_exptimes(exptime: List[datetime], fps: float) -> (List[datetime], List[int]):
    """Normalize and filter exptime array to return valid exptimes and their corresponding frame numbers."""
    base_time = exptime[0]
    filtered_frame_numbers = []
    filtered_exptime = []
    ignored_frames = 0  # Count of ignored frames due to abnormal values

    for i, exp_time in enumerate(exptime):
        frame_number = (exp_time - base_time).total_seconds() * fps
        # Exclude abnormal values
        if i > 0:
            expected_frame_diff = frame_number - filtered_frame_numbers[-1]
            if expected_frame_diff < 0 or expected_frame_diff > 4 + 3 * ignored_frames:
                ignored_frames += 1
                continue
            ignored_frames = 0  # Reset the count as this frame is normal
        filtered_frame_numbers.append(frame_number)
        filtered_exptime.append(exp_time)

    return filtered_exptime, filtered_frame_numbers

# The rest of the code remains the same
# Example data and usage would also be the same as before


def linear_regression_with_fixed_slope(frame_numbers: List[int], exptime: List[datetime], fps: float) -> float:
    """Perform linear regression with a fixed slope of 1/fps and return the y-intercept."""
    times_in_seconds = [(exp_time - exptime[0]).total_seconds() for exp_time in exptime]
    slope = 1 / fps
    intercept = np.mean(times_in_seconds) - slope * np.mean(frame_numbers)
    return intercept

# Example usage
fps = 30  # Example frame rate

# Example data (replace this with actual exptime data)
exptime_example = [datetime(2020, 1, 1) + timedelta(seconds=i*1/fps) for i in range(60)]

# Normalize and filter exptime
normalized_exptime, frame_numbers = normalize_and_filter_exptimes(exptime_example, fps)

# Perform linear regression with fixed slope
y_intercept = linear_regression_with_fixed_slope(frame_numbers, normalized_exptime, fps)

y_intercept  # This will be the y-intercept from the regression



-1.1102230246251565e-16

In [113]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

# Prepare the data
fps = 30  # Example frame rate
exptime_example = [datetime(2020, 1, 1) + timedelta(seconds=i*1/fps) for i in range(60)]

# Normalize and filter exptime
normalized_exptime, frame_numbers = normalize_and_filter_exptimes(exptime_example, fps)

# Calculate y-intercept for regression line
y_intercept = linear_regression_with_fixed_slope(frame_numbers, normalized_exptime, fps)

# Calculate regression line
regression_line = [y_intercept + (1/fps) * frame for frame in frame_numbers]

# Calculating errors (difference between actual exptime and regression line)
errors = [(exp_time - normalized_exptime[0]).total_seconds() - reg_time for exp_time, reg_time in zip(normalized_exptime, regression_line)]

# Preparing the plot
output_notebook()  # Set up to display plot in notebook
p = figure(title="Exptime vs Regression Line Difference", x_axis_label='Frame Number', y_axis_label='Difference in seconds')

# Plotting errors
p.circle(frame_numbers, errors, legend_label="Errors", fill_color="blue", size=8)

# Plotting the ignored values (if any)
ignored_indices = set(range(len(exptime_example))) - set([exptime_example.index(exp_time) for exp_time in normalized_exptime])
for idx in ignored_indices:
    ignored_frame_number = (exptime_example[idx] - exptime_example[0]).total_seconds() * fps
    if idx > 0:
        ignored_frame_number = frame_numbers[idx-1] + 1  # Frame number of ignored value
    p.cross([ignored_frame_number], [0], legend_label="Ignored Values", fill_color="red", size=8)

# Show the plot
show(p)


Loading BokehJS ...

In [114]:
def drawExptimeError(exptime: List[datetime], fps: float) -> figure:
    """Analyzes and plots the difference between exptime[] and the regression line.
    
    Args:
    exptime (List[datetime]): List of datetime objects representing the exposure times.
    fps (float): The frame rate of the video.

    Returns:
    figure: Bokeh figure object representing the plot.
    """
    # Normalize and filter exptime
    normalized_exptime, frame_numbers = normalize_and_filter_exptimes(exptime, fps)

    # Calculate y-intercept for regression line
    y_intercept = linear_regression_with_fixed_slope(frame_numbers, normalized_exptime, fps)

    # Calculate regression line
    regression_line = [y_intercept + (1/fps) * frame for frame in frame_numbers]

    # Calculating errors (difference between actual exptime and regression line)
    errors = [(exp_time - normalized_exptime[0]).total_seconds() - reg_time for exp_time, reg_time in zip(normalized_exptime, regression_line)]

    # Preparing the plot
    p = figure(title="Exptime vs Regression Line Difference", x_axis_label='Frame Number', y_axis_label='Difference in seconds')

    # Plotting errors
    p.circle(frame_numbers, errors, legend_label="Errors", fill_color="blue", size=8)

    # Plotting the ignored values (if any)
    ignored_indices = set(range(len(exptime))) - set([exptime.index(exp_time) for exp_time in normalized_exptime])
    for idx in ignored_indices:
        ignored_frame_number = (exptime[idx] - exptime[0]).total_seconds() * fps
        if idx > 0:
            ignored_frame_number = frame_numbers[idx-1] + 1  # Frame number of ignored value
        p.cross([ignored_frame_number], [0], legend_label="Ignored Values", fill_color="red", size=8)

    return p

# Example usage:
# fps = 30  # Example frame rate
# exptime_example = [datetime(2020, 1, 1) + timedelta(seconds=i*1/fps) for i in range(60)]
# plot = drawExptimeError(exptime_example, fps)
# show(plot)  # Display the plot



In [115]:
fps = 30  # フレームレート
exptime_example = [datetime(2020, 1, 1) + timedelta(seconds=i*1/fps) for i in range(60)]  # サンプルデータ
plot = drawExptimeError(exptime_example, fps)
show(plot)  # グラフを表示